### Preprocessing
this code is used to create the dataset for the machine learning model au-bs model


In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from scipy import interpolate
from matplotlib import pyplot as plt
from IPython.display import display
import os
from pathlib import Path

path = "C:/Users/Tony/Documents/TestData/"
list_participants_id_path = [f.path for f in os.scandir(path) if f.is_dir() and "." not in f.name]
#list_of_openface_csvs = [f.path for f in os.scandir(participant_path) if f.is_file and f.name.endswith('.csv')]

def low_pass_filter(csv_file: str, window_ms: int=100) -> pd.DataFrame:
    '''
    low pass filter the data to filter noise using a sliding windows of 100ms by default
    outputs a new csv file
    '''
    window_size = int(window_ms*60/1000)
    bs_df = pd.read_csv(csv_file, sep=', ')
    #drop the frame columnm, as it is not useful anymore
    rolling = bs_df.drop("frame", axis=1).rolling(window_size).mean()
    return rolling
    
def rreplace(s, old, new, occurence):
    '''
    find 'old' and replace by 'new'
    it does so 'occurrence' times
    starting at the end of 's'
    '''
    li = s.rsplit(old, occurence)
    return new.join(li)

def processing_milliseconds(s):
    '''
    converts milliseconds base 60 into actual milliseconds base 10
    does not work with negative time
    '''
    li = s.rsplit(':', 1)
    millisec = float(li[1])
    #divide by 60 and multiply by 1sec in microseconds
    millisec = (millisec/60)*1e6
    if len(str(int(millisec))) < 6:
        li[1] = ':0'+str(millisec)
    else:
        li[1] = ':'+str(millisec) 
    return ''.join(li)

def compute_framerate(data):
    n_seconds = np.sum(np.diff(data))
    n_frames = data.shape[0]-1
    framerate = n_frames / n_seconds
    return framerate

def find_corresponding_blendshape_csv(participants_id_path, filepath):
    filename = os.path.basename(filepath)
    path_to_bs_csv = participants_id_path+"/csv_whole/individual"
    emotion_intensity = filename.split('!')[1]
    emotion_number = filename.split('!')[2]
    file = [f.path for f in os.scandir(path_to_bs_csv) if emotion_intensity in f.name and emotion_number in f.name]
    return file[0]

def cross_corr_with_savgol_filter(signal1, signal2, window_size):
    '''
    returns the correlation and the lags in a tuple
    '''
    #Before cross correlation, a savgol filter is applied to smooth high frequencies
    y_au_filtered = signal.savgol_filter(signal1, window_size, 1)
    y_bs_filtered = signal.savgol_filter(signal2.to_numpy().flatten(), window_size, 1)
    correlation = signal.correlate(y_au_filtered-np.mean(y_au_filtered), y_bs_filtered-np.mean(y_bs_filtered), mode="full") #substracting the mean makes computing more accurate
    #The lag is the refers to how far the series are offset
    lags = signal.correlation_lags(len(y_au_filtered), len(y_bs_filtered), mode="full")
    return [correlation, lags]

def plot_for_testing_purposes(blendshape_file, au_file, au_name, bs_name):
    """
    Plot an action unit and blendshape with specified path of both files
    Used to plot often AU26_r and JawOpen
    """
    action_unit = pd.read_csv(au_file, sep=', ', engine="python")
    blendshape = pd.read_csv(blendshape_file, sep=',')
    x_au = action_unit["timestamp"]
    y_au = action_unit[au_name]
    x_bs = blendshape["Timecode"]
    y_bs = blendshape[bs_name]
    y_au/=5 # normalization between 0 and 1

    # 21:52:40:24.937 --> 21:52:40:024937
    # x2 = x2.apply(processing_milliseconds)
    # x2 = x2.str.split('.').str[0]
    # 2022-01-01 14:38:12.133575
    x_bs = pd.to_datetime(x_bs, format="%Y-%m-%d %H:%M:%S.%f")
    x_bs -= x_bs[0]

    #Low pass filter of a windows size of 100
    y1_tmp = y_au.rolling(6).mean().dropna()
    y2_tmp = y_bs.rolling(6).mean().dropna()

    y_au = y_au.rolling(6).mean().fillna(np.mean(y1_tmp))
    y_bs = y_bs.rolling(6).mean().fillna(np.mean(y2_tmp))

    #y_au = y_au.iloc[:len(y_bs)].reset_index(drop=True)

    #--Cross correlation--
    correlation, lags = cross_corr_with_savgol_filter(y_au, y_bs, 20)
    #We get the lag at the peak of the correlation, when both signal correlate the best
    lag = lags[np.argmax(abs(correlation))]
    #Plottting the signal by slicing values. The number of sliced values is "lag"
    fig, ax1 = plt.subplots()
    fig2, ax2 = plt.subplots()
    #y_bs = y_bs.iloc[abs(lag):].reset_index(drop=True)
    y_bs_resampled=signal.resample(y_bs, 100)
    y_au_resampled=signal.resample(y_au, 100)
    ax1.plot(y_au, color="green", label=au_name)
    ax1.plot(y_bs, color="blue", label=bs_name)
    ax2.plot(y_au_resampled, label=au_name + 'Resampled')
    ax2.plot(y_bs_resampled, label=bs_name + 'Resampled')
    ax1.legend(loc="upper right")
    ax1.title.set_text(Path(blendshape_file).name)
    ax2.legend(loc="upper right")
    ax2.title.set_text(Path(blendshape_file).name)
    #display(fig)
    #plt.savefig("resampled/" + Path(blendshape_file).name + ".png")
    
def find_all_bs_csvs(row):
    participant_id = row.split("_")[0]
    folders_in_between = "/csv_whole/individual/"
    suffix = "_whole.csv"
    return os.path.join(path, participant_id + folders_in_between + row + suffix)


def find_outliers(threshold):
    """
    find the files that have a frame difference over the threshold variable
    stores them in a csv
    """
    lag_csv = pd.read_csv(path+ "/AU26_r-JawOpen-lagMeasure.csv" , sep=',')
    all_bs_csvs = lag_csv["blendshape_scriptID"].apply(find_all_bs_csvs).values.tolist()
    all_participants = lag_csv["participant_id"].drop_duplicates(keep="first")
    all_au_csvs = all_participants.apply(lambda x: [f.path for f in os.scandir(x) if f.is_file and f.name.endswith('.csv')]) #find all au csv for each folder named as the participants
    all_au_csvs = all_au_csvs.explode().reset_index(drop=True).values.tolist() #explode the lists inside cells in rows
    frame_diff = lag_csv["n_frames_OF"] - lag_csv["n_frame_ARKIT"]

    #outlier threshold: if the frame difference between both file is more than 60 frames, then we select its index
    list_frame_list_sup_1s = frame_diff[abs(frame_diff) > threshold].index.values.tolist()
    framerate_au = lag_csv["average_frame_rate_OF"].values.tolist()
    framerate_bs = lag_csv["average_frame_rate_ARKIT"].values.tolist()
    bs_csvs = [all_bs_csvs[index] for index in list_frame_list_sup_1s]
    au_csvs = [all_au_csvs[index] for index in list_frame_list_sup_1s]
    framerate_au = [framerate_au[index] for index in list_frame_list_sup_1s]
    framerate_bs = [framerate_bs[index] for index in list_frame_list_sup_1s]
    outlier_frame = pd.DataFrame(columns=["index_in_lagMeasure_csv", "frame_diff_(au-bs)", "openface_framerate", "arkit_framerate","action_unit_csv", "blend_shape_csv"])
    for au, bs, idx, f_au, f_bs in zip(au_csvs, bs_csvs, list_frame_list_sup_1s, framerate_au, framerate_bs):
        au_csv = pd.read_csv(au)
        bs_csv = pd.read_csv(bs)
        frame_diff = au_csv.shape[0] - bs_csv.shape[0]
        dictionary = {"index_in_lagMeasure_csv": idx, "frame_diff_(au-bs)": frame_diff, "openface_framerate": f_au, "arkit_framerate": f_bs ,"action_unit_csv": Path(au).name, "blend_shape_csv": Path(bs).name}
        outlier_frame = pd.concat([outlier_frame, pd.DataFrame.from_records([dictionary])], ignore_index=True)
    outlier_frame.to_csv("outliers.csv", index=False)


def low_pass_filter_and_lag_slicing(bs_dt, au_dt, lag, window_size):
    """
    Apply a low pass filter of size "window" on both dataframe au_dt and bs_dt
    Slice the lag of the bs dataset, computed earlier and available in the lagMeasure.csv
    cut the dataframes so that both files have the same number lines
    """
    #####Low pass filter
    #low pass without the nan values
    au_tmp = au_dt.rolling(window_size).mean().dropna()
    bs_tmp = bs_dt.rolling(window_size).mean().dropna()
    #low pass filter + replacing the nan values by the mean of tmp values
    au_dt = au_dt.rolling(window_size).mean().fillna(np.mean(au_tmp))
    bs_dt = bs_dt.rolling(window_size).mean().fillna(np.mean(bs_tmp))
    #Slice the lag at the start of the bs file
    bs_dt = bs_dt.iloc[abs(lag):].reset_index(drop=True)

    #Slice the end of files so that both files math in length
    if au_dt.shape[0] > bs_dt.shape[0]: #if the number of lines of au file is greater than the bs file
        au_dt = au_dt.iloc[: bs_dt.shape[0]]
    else:
        bs_dt = bs_dt.iloc[: au_dt.shape[0]]
    return [au_dt, bs_dt]

def remove_outliers(list_val, lag_csv, threshold):
    '''
    remove outliers from the list "list_val" where the frame difference between au and blendshape is > to "threshold"
    '''
    frame_diff = lag_csv["n_frames_OF"] - lag_csv["n_frame_ARKIT"]
    list_frame_list_sup_1s = frame_diff[abs(frame_diff) <= threshold].index.values.tolist()
    return [list_val[index] for index in list_frame_list_sup_1s]

def create_au_bs_dataset():
    """
    this methods merge all the data from blendshapes, to action units and other data extracted with OpenFace that are availabe in the au.csv (gaze, 3D landmarks etc.)
    It creates a new csv file that will be the foundation for the au to bs machine learning model
    the dataset is called giga_dataset
    WARNING: the computation takes time
    """
    #Open the csv file
    lag_csv = pd.read_csv(path+ "/AU26_r-JawOpen-lagMeasure.csv" , sep=',')
    all_bs_csvs = lag_csv["blendshape_scriptID"].apply(find_all_bs_csvs).values.tolist()
    lag_vals = lag_csv["max_lag"].values.tolist()
    #load all the csvs
    all_participants = lag_csv["participant_id"].drop_duplicates(keep="first")
    all_au_csvs = all_participants.apply(lambda x: [f.path for f in os.scandir(x) if f.is_file and f.name.endswith('.csv')]) #find all au csv for each folder named as the participants
    all_au_csvs = all_au_csvs.explode().reset_index(drop=True).values.tolist() #explode the lists inside cells in rows

    au_csvs = remove_outliers(all_au_csvs, lag_csv, 60)
    bs_csvs = remove_outliers(all_bs_csvs, lag_csv, 60)
    lag_vals_without_outliers = remove_outliers(lag_vals, lag_csv, 60)
    #print('\n'.join('{}: {}'.format(*k) for k in enumerate(au_csvs))) #print the data
    giga_dataset = pd.DataFrame()
    list_of_df = []
    for au_path, bs_path, lag in zip(au_csvs, bs_csvs, lag_vals_without_outliers):
        bs_dt = pd.read_csv(bs_path, index_col=0) #index_col=0 removes the unnamed: 0 column
        au_dt = pd.read_csv(au_path, sep=", ", engine='python')
        #drop unecessary columns
        au_dt.drop(["frame", "face_id", "timestamp", "confidence", "success"], axis=1, inplace=True)
        au_dt = au_dt.loc[:, ~au_dt.columns.str.endswith("_c")]
        bs_dt = bs_dt.drop(["Timecode", "BlendShapeCount"], axis=1).reset_index(drop=True) #index starts at 0 with reset_index(drop=True)
        #divide by 5 to normalize au value between 0 and 1
        au_dt.loc[:, au_dt.columns.str.endswith("_r")] /= 5 
        au_dt, bs_dt = low_pass_filter_and_lag_slicing(bs_dt, au_dt, lag, 6)

        #########Uncomment to see the AU26_r and JawOpen plot, and see how it goes
        # fig, ax1 = plt.subplots()
        # ax1.plot(au_dt["AU26_r"], color="green", label="AU26_r")
        # ax1.plot(bs_dt["JawOpen"], color="blue", label="JawOpen")
        # ax1.legend(loc="upper right")
        # display(fig)
        
        #Add script id column at the start of the dataframe
        script_n = [Path(au_path).name.rsplit('!', 1)[0]] * au_dt.shape[0]
        script_name_col = pd.DataFrame(script_n, columns=["script_id"])

        list_of_df.append(pd.concat([script_name_col, au_dt, bs_dt], axis=1))

    giga_dataset = pd.concat(list_of_df)
    giga_dataset.to_csv("giga_dataset.csv",  index=False)
#create_au_bs_dataset()




#############Plot all the au/bs over all the scripts of all participants##############################
#############You need to choose au and bs (e.g: AU26_r and JawOpen)
# for participant in list_participants_id_path:
#     list_of_openface_csvs = [f.path for f in os.scandir(participant) if f.is_file and f.name.endswith('.csv')]
#     for openface_csv in list_of_openface_csvs:
#         blendshape_csv = find_corresponding_blendshape_csv(participant, openface_csv)
#         for bs, au in blendshape_au_dictionary.items():
#             plot_for_testing_purposes(blendshape_csv, openface_csv, au, bs)


#################Determine which windows size to choose with savgol filter###############################
    # fig, axs = plt.subplots(nrows=10, ncols=2, figsize=(25, 50))
    # for i, ax in zip(range(1, 20), axs.flat):
    #     y1= signal.savgol_filter(y1, i, 0)
    #     y2 = signal.savgol_filter(y2, i, 0)
    #     ax.plot(x1, y1, color="green", label=ActionUnitName)
    #     ax.plot(x2.dt.total_seconds(), y2, color="blue", label=blendshapeName)
    #     ax.legend(loc="upper right")
    #     ax.title.set_text(blendshape_file + "\nsavgol size: "+ str(i))

In [ ]:
def plot_JawOpen_with_lag_corrected():
    lag_csv = pd.read_csv(path+ "/AU26_r-JawOpen-lagMeasure.csv" , sep=',')
    all_bs_csvs = lag_csv["blendshape_scriptID"].apply(find_all_bs_csvs).values.tolist()
    lag_vals = lag_csv["max_lag"].values.tolist()
    #load all the csvs
    all_participants = lag_csv["participant_id"].drop_duplicates(keep="first")
    all_au_csvs = all_participants.apply(lambda x: [f.path for f in os.scandir(x) if f.is_file and f.name.endswith('.csv')]) #find all au csv for each folder named as the participants
    all_au_csvs = all_au_csvs.explode().reset_index(drop=True).values.tolist() #explode the lists inside cells in rows

    au_csvs = remove_outliers(all_au_csvs, lag_csv, 60)
    bs_csvs = remove_outliers(all_bs_csvs, lag_csv, 60)
    lag_vals_without_outliers = remove_outliers(lag_vals, lag_csv, 60)
    for au_path, bs_path, lag in zip(au_csvs, bs_csvs, lag_vals_without_outliers):
        bs_dt = pd.read_csv(bs_path, index_col=0) #index_col=0 removes the unnamed: 0 column
        au_dt = pd.read_csv(au_path, sep=", ", engine='python')
        #drop unecessary columns
        au_dt.drop(["frame", "face_id", "timestamp", "confidence", "success"], axis=1, inplace=True)
        au_dt = au_dt.loc[:, ~au_dt.columns.str.endswith("_c")]
        bs_dt = bs_dt.drop(["Timecode", "BlendShapeCount"], axis=1).reset_index(drop=True) #index starts at 0 with reset_index(drop=True)
        #divide by 5 to normalize au value between 0 and 1
        au_dt.loc[:, au_dt.columns.str.endswith("_r")] /= 5 
        au_dt, bs_dt = low_pass_filter_and_lag_slicing(bs_dt, au_dt, lag, 6)
        fig, ax = plt.subplots()
        ax.plot(au_dt["AU26_r"], color="green", label="AU 26")
        ax.plot(bs_dt["JawOpen"], color="blue", label="JawOpen")
        ax.legend(loc="upper right")
        bs_name = Path(bs_path).name.rsplit("_", 1)[0]
        ax.set_xlabel("frame over time")
        ax.title.set_text(bs_name + "  lag corrected")
        plt.savefig("JawOpen_AU26_corrected/" + bs_name + ".png")
plot_JawOpen_with_lag_corrected()